In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset


In [ ]:
import json

data = [
  {"prompt": "What is the current temperature in the living room?", "response": "22°C"},
  {"prompt": "Is the front door currently locked?", "response": "true"},
  {"prompt": "When was the last time the kitchen lights were turned on?", "response": "2023-12-25 18:30:00"},
  {"prompt": "What is the current humidity level in the bedroom?", "response": "45%"},
  {"prompt": "How much energy did the dishwasher consume during its last cycle?", "response": "0.8 kWh"},
  {"prompt": "Is the garage door currently open?", "response": "false"},
  {"prompt": "What is the current battery level of the smoke detector in the hallway?", "response": "85%"},
  {"prompt": "When was the last time the sprinkler system was activated?", "response": "2023-12-20 10:00:00"},
  {"prompt": "What is the current air quality index in the living room?", "response": "75"},
  {"prompt": "How many times has the doorbell rung today?", "response": "3"},
  {"prompt": "When was the last time the security system was armed?", "response": "2023-12-25 22:00:00"},
  {"prompt": "What is the current water temperature of the hot water heater?", "response": "60°C"},
  {"prompt": "How many hours has the air conditioner been running today?", "response": "4"},
  {"prompt": "Is the refrigerator door currently open?", "response": "false"},
  {"prompt": "When was the last time the oven was used?", "response": "2023-12-23 17:00:00"},
  {"prompt": "What is the current CO2 level in the bedroom?", "response": "400 ppm"},
  {"prompt": "How much energy has the washing machine consumed this month?", "response": "30 kWh"},
  {"prompt": "Is the fireplace currently on?", "response": "false"},
  {"prompt": "When was the last time the blinds in the living room were closed?", "response": "2023-12-25 16:00:00"},
  {"prompt": "What is the current temperature outside?", "response": "10°C"},
   {
    "prompt": "Q: How many windows are currently open?\nLet's think step by step:\nFirst, check each room. There are 2 windows open in the bedroom and 1 in the living room. That makes a total of 3 windows.",
    "response": "3"
  },
  # for chain of thought prompting
  {
    "prompt": "Q: How much energy did the air conditioner use today?\nLet's think step by step:\n- First, let's check how long the air conditioner has been running.\n- The air conditioner has been running for 5 hours today.\n- Next, we check the power consumption rate of the air conditioner.\n- The air conditioner uses 1.2 kWh per hour.\n- Now, multiply the running hours by the energy usage rate: 5 × 1.2 = 6 kWh.\nSo, the air conditioner used 6 kWh of energy today.",
    "response": "6 kWh"
  },
  {
    "prompt": "Q: When was the last time the sprinkler system was activated?\nLet's break it down:\n- First, we need to check the activity log for the sprinkler system.\n- The last activation was recorded on December 20, 2023, at 10:00 AM.\nSo, the sprinkler system was last activated on December 20, 2023, at 10:00 AM.",
    "response": "December 20, 2023, at 10:00 AM"
  },
  {
    "prompt": "Q: What is the current battery level of the smoke detector in the hallway?\nLet's think step by step:\n- First, we check the battery status of the smoke detector in the hallway.\n- The current battery level is 85%.\nSo, the smoke detector's battery is 85% charged, which means it is functioning properly.",
    "response": "85%"
  },
  {
    "prompt": "Q: How much energy did the dishwasher consume during its last cycle?\nLet's think step by step:\n- First, check how long the last dishwasher cycle ran.\n- The dishwasher cycle lasted for 1.5 hours.\n- Next, we check the power consumption rate of the dishwasher.\n- The dishwasher consumes 0.8 kWh per hour.\n- Now, multiply the time by the energy rate: 1.5 × 0.8 = 1.2 kWh.\nSo, the dishwasher consumed 1.2 kWh during its last cycle.",
    "response": "1.2 kWh"
  },
  {
    "prompt": "Q: Is the security system currently armed?\nLet's think step by step:\n- First, check the current status of the security system.\n- The security system log shows that it was armed at 10:00 PM last night.\n- Since no disarm action has been recorded since then, the system is still armed.\nSo, yes, the security system is currently armed.",
    "response": "Yes"
  },
  {
    "prompt": "Q: Is the garage door currently open?\nLet's break it down:\n- First, we need to check the current status of the garage door sensor.\n- The sensor indicates that the door is closed.\nSo, no, the garage door is not currently open.",
    "response": "No"
  },
  {
    "prompt": "Q: When was the last time the oven was used?\nLet's think step by step:\n- First, check the activity log of the oven.\n- The log shows that the oven was last turned on at 5:00 PM on December 23, 2023.\nSo, the oven was last used at 5:00 PM on December 23, 2023.",
    "response": "December 23, 2023, at 5:00 PM"
  },
  {
    "prompt": "Q: What is the current water temperature of the hot water heater?\nLet's break it down:\n- First, check the current temperature reading from the hot water heater.\n- The water heater is currently set to 60°C.\nSo, the current water temperature is 60°C.",
    "response": "60°C"
  },
  {
    "prompt": "Q: When were the blinds in the living room last closed?\nLet's think step by step:\n- First, check the log of the smart blinds in the living room.\n- The blinds were last closed at 4:00 PM on December 25, 2023.\nSo, the blinds were last closed at 4:00 PM on December 25, 2023.",
    "response": "December 25, 2023, at 4:00 PM"
  },
  {
    "prompt": "Q: What is the current CO2 level in the bedroom?\nLet's break it down:\n- First, check the CO2 sensor in the bedroom.\n- The sensor is reading a CO2 level of 400 ppm.\nSo, the current CO2 level in the bedroom is 400 ppm, which is within the normal range.",
    "response": "400 ppm"
  },
  {
    "prompt": "Q: What is the current air quality index in the living room?\nLet's think step by step:\n- First, check the air quality sensor in the living room.\n- The air quality index (AQI) is reading 75.\n- According to AQI standards, this is in the 'Moderate' range.\nSo, the air quality in the living room is moderate with an AQI of 75.",
    "response": "75"
  },
  {
    "prompt": "Q: Is the refrigerator door currently open?\nLet's break it down:\n- First, check the current status of the refrigerator door sensor.\n- The sensor indicates that the door is closed.\nSo, no, the refrigerator door is not currently open.",
    "response": "No"
  },
  {
    "prompt": "Q: How much energy has the washing machine consumed this month?\nLet's think step by step:\n- First, check the washing machine’s energy log for this month.\n- The washing machine has consumed 30 kWh so far this month.\nSo, the washing machine has used 30 kWh of energy this month.",
    "response": "30 kWh"
  },
  {
    "prompt": "Q: How many devices are currently connected to the smart home system?\nLet's break it down:\n- First, check the network or hub to see how many devices are currently online.\n- There are 15 devices currently connected to the smart home network.\nSo, 15 devices are currently online.",
    "response": "15 devices"
  },
  {
    "prompt": "Q: How many lights are currently turned on in the house?\nLet's think step by step:\n- First, check the status of all connected smart lights.\n- 2 lights are on in the kitchen, and 1 light is on in the living room.\n- So, there are 3 lights currently on in the house.\nSo, 3 lights are currently turned on.",
    "response": "3 lights"
  }

]

with open('iot_dataset.json', 'w') as f:
    json.dump(data, f)


In [ ]:
dataset = load_dataset('json',data_files='iot_dataset.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainer ,Seq2SeqTrainingArguments
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer =AutoTokenizer.from_pretrained(model_name)



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def preprocess_function(examples):
    inputs = examples['prompt']
    targets = examples['response']
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    # Ensure the lengths are the same
    if len(model_inputs["input_ids"]) != len(labels["input_ids"]):
        raise ValueError(f"Input and label lengths do not match: {len(model_inputs['input_ids'])} vs {len(labels['input_ids'])}")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs




In [ ]:
from datasets import load_dataset, DatasetDict

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Inspect the dataset
print(tokenized_dataset['train'][0])  # Inspect a sample



# Split dataset if necessary
if 'validation' not in tokenized_dataset:
    train_valid_split = tokenized_dataset['train'].train_test_split(test_size=0.1)
    tokenized_dataset = DatasetDict({
        'train': train_valid_split['train'],
        'validation': train_valid_split['test']
    })

# Verify the splits
print(tokenized_dataset)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=3,
    predict_with_generate=True,
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
)

# Start training
trainer.train()


Map:   0%|          | 0/36 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'prompt': 'What is the current temperature in the living room?', 'response': '22°C', 'input_ids': [363, 19, 8, 750, 2912, 16, 8, 840, 562, 58, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [1630, 1956, 254, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Epoch,Training Loss,Validation Loss
1,No log,48.681328
2,No log,44.627777
3,No log,43.387573


TrainOutput(global_step=24, training_loss=43.100067138671875, metrics={'train_runtime': 546.525, 'train_samples_per_second': 0.176, 'train_steps_per_second': 0.044, 'total_flos': 16434176458752.0, 'train_loss': 43.100067138671875, 'epoch': 3.0})

In [ ]:
model.save_pretrained('./fine-tuned-flan-t5-iot')
tokenizer.save_pretrained('./fine-tuned-flan-t5-iot')

('./fine-tuned-flan-t5-iot/tokenizer_config.json',
 './fine-tuned-flan-t5-iot/special_tokens_map.json',
 './fine-tuned-flan-t5-iot/spiece.model',
 './fine-tuned-flan-t5-iot/added_tokens.json',
 './fine-tuned-flan-t5-iot/tokenizer.json')

In [ ]:
inputs = tokenizer("What is the current temperature reported by the greenhouse sensor?", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


0 °C


In [ ]:
inputs = tokenizer("What is the current temperature reported by the greenhouse sensor?", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


0 °C


In [ ]:
inputs = tokenizer("Is the fireplace currently on?", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The fireplace is still on.


In [ ]:
inputs = tokenizer("How many times has the doorbell rung today?", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

ten


In [ ]:
# Prepare the prompt for zero-shot inference

prompt = "Is the front door currently locked?"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate output from the model
output = model.generate(**inputs)

# Decode the output
response = tokenizer.decode(output[0], skip_special_tokens=True)
print("Zero-shot response:", response)

Zero-shot response: no


In [ ]:
# Define the prompt with one-shot example
prompt = """
Example:
Q: Is the front door currently locked?
A: True

Now answer:
Q: Is the garage door currently open?
A:"""

# Tokenize and generate output
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs)
response = tokenizer.decode(output[0], skip_special_tokens=True)
print("One-shot response:", response)

One-shot response: No


In [ ]:
# Define the prompt with few-shot examples
prompt = """
Examples:
Q: Is the front door currently locked?
A: True

Q: When was the last time the kitchen lights were turned on?
A: 2023-12-25 18:30:00

Q: What is the current humidity level in the bedroom?
A: 45%

Now answer:
Q: Is the garage door currently open?
A:"""

# Tokenize and generate output
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs)
response = tokenizer.decode(output[0], skip_special_tokens=True)
print("Few-shot response:", response)

Few-shot response: No


In [ ]:
# chain of thought prompting

prompt = """
Q: How many windows are currently open?
Let's think step by step:
- There are 2 windows open in the bedroom.
- There is 1 window open in the living room.
So, in total, there are 3 windows open.

Q: What is 23 + 19?
Let's break it down step by step:
- First, add 20 to 23 to get 43.
- Then, subtract 1 from 43 to get 42.
So, the answer is 42.

Q: How much is 12 + 15?
"""

# Tokenize and generate input
inputs = tokenizer(prompt, return_tensors="pt")
output = model.generate(**inputs)
response = tokenizer.decode(output[0], skip_special_tokens=True)
print("Few-shot response:", response)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Few-shot response: Let's break it down step by step: 12 + 15 = 12 + 15. So,


In [ ]:
# Chain of Thought prompting

prompt = """
Q: How many windows are currently open?
Let's think step by step:
- There are 2 windows open in the bedroom.
- There is 1 window open in the living room.
So, in total, there are 3 windows open.


Q: How many windows are open in bedroom ?
"""

inputs =tokenizer(prompt,return_tensors="pt")
output =model.generate(**inputs)
response = tokenizer.decode(output[0],skip_special_tokens=True)
print("following is response",response)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


following is response There are 2 windows in the bedroom. There are 1 window in the living room. So,


In [ ]:
dd = []
dd.append("device-id")
dd.append("device-name")
dd.append("action")
dd.append("time")
print(dd)

['device-id', 'device-name', 'action', 'time']


In [ ]:
dd.append([1,"fridge","opened by skand","timeing was 12 pm"])
dd.append([2,"AC","Ac was closed by itself","time do not know"])
dd.append([3,"tv","someone broke the tv","around 1 pm"])
dd.append([1,"fridge","closed by kartikey","around 1 am"])
print(dd)





['device-id', 'device-name', 'action', 'time', [1, 'fridge', 'opened by skand', 'timeing was 12 pm'], [2, 'AC', 'Ac was closed by itself', 'time do not know'], [1, 'fridge', 'opened by skand', 'timeing was 12 pm'], [2, 'AC', 'Ac was closed by itself', 'time do not know'], [3, 'tv', 'someone broke the tv', 'around 1 pm'], [1, 'fridge', 'closed by kartikey', 'around 1 am']]


In [ ]:
import pandas as pd
import numpy as np

# Create a DataFrame with named columns
data = {
    'a': np.random.rand(10),
    'b': np.random.rand(10),
    'c': np.random.rand(10),
    'd': np.random.rand(10)
}
df = pd.DataFrame(data)

print(df)

          a         b         c         d
0  0.941745  0.578464  0.663583  0.152333
1  0.147970  0.453929  0.720844  0.936473
2  0.069154  0.757662  0.654975  0.834616
3  0.188173  0.147196  0.084091  0.276709
4  0.018959  0.418186  0.794336  0.097255
5  0.960065  0.353291  0.659867  0.925461
6  0.875143  0.176070  0.599552  0.455794
7  0.443537  0.525922  0.684938  0.637891
8  0.030747  0.192463  0.134609  0.695478
9  0.275135  0.831272  0.311929  0.660853


In [ ]:
data ={"device-id":[],"device-name":[],"action":[],"time":[]}
df = pd.DataFrame(data)
print(df)

Empty DataFrame
Columns: [device-id, device-name, action, time]
Index: []


In [ ]:
print(df.columns)
print(len(df))
print(df.shape)
df.append([1,"fridge","opened by skand","timeing was 12 pm"])
df.append([2,"AC","Ac was closed by itself","time do not know"])
df.append([3,"tv","someone broke the tv","around 1 pm"])
df.append([1,"fridge","closed by kartikey","around 1 am"])

Index(['device-id', 'device-name', 'action', 'time'], dtype='object')
0
(0, 4)


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
l=[-25.82501623,	-30.07508668,
-37.42527197,	-32.16880981,
-35.43773516,	-45.03983221,
-28.52476296,	-39.18457302,
-32.67371628,	-35.15226686,
-48.38727906,	-43.788397,
-21.41928125,	-52.5096872,
-32.23220943,	-38.4035275,
-32.67784108,	-42.47122021,
-35.43773516,	-39.55945656]

a=0
b=0
for i in range(0,len(l)):
  if(i%2==0):
    a+=l[i]
  else:
    b+=l[i]
print(2*a/len(l))
print(2*b/len(l))

-33.004084858
-39.835285705000004
